In [1]:
print("SDdd")

SDdd


In [2]:
import folium
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [243]:
# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection("MODIS/061/MOD11A1")

In [244]:
# Initial date of interest (inclusive).
i_date = '2019-07-01'

# Final date of interest (exclusive).
f_date = '2020-07-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [245]:
lst

In [246]:
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

In [247]:
lst_img

In [248]:
# Define the urban location of interest as a point
#lon = -73.974691
#lat = 40.741374
lat = 33.7488
lon = -84.3877

poi = ee.Geometry.Point(lon, lat)

# Define a region of interest with a buffer zone of 100 km around point.
roi = poi.buffer(1e5)

In [249]:
def add_ee_layer(my_map, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(my_map)
    my_map.add_child(folium.LayerControl())

In [250]:
lst_vis_params = {
    'min': 0, 'max': 30,
    'palette': ['white', 'blue', 'green', 'yellow', 'orange', 'red']}

In [251]:
lst_vis_params

{'min': 0,
 'max': 30,
 'palette': ['white', 'blue', 'green', 'yellow', 'orange', 'red']}

In [252]:
my_map = folium.Map(location=[lat, lon], zoom_start=10)

add_ee_layer(my_map, lst_img, lst_vis_params, 'Land Surface Temperature')

In [253]:
my_map

In [254]:
# Save to GeoTIFF
task = ee.batch.Export.image.toDrive(image=lst_img,
                                     description='lst_atlanta_'+str(i_date) + str(f_date),
                                     scale=30,
                                     region=roi,
                                     fileNamePrefix='lst_atlanta_'+str(i_date) + str(f_date),
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')

In [255]:
task.start()

In [256]:
task.status()

{'state': 'READY',
 'description': 'lst_atlanta_2019-07-012020-07-01',
 'creation_timestamp_ms': 1683044832048,
 'update_timestamp_ms': 1683044832048,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '4KQNDZEWVQH7TPQGTLQDJV7P',
 'name': 'projects/earthengine-legacy/operations/4KQNDZEWVQH7TPQGTLQDJV7P'}